## Fixations numerations fix (for aDDM_toolbox)

+ Change the numeration of trials once they are separated in Like and Dislike conditions (avoid breaks in trial number)
+ Calculate the gaze time for each trial

In [1]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from copy import copy as copy
import operator as operator
import pylab

# More condition data 

In [2]:
#Input has the fixation and expdat with breaks in trial numerations (due to separation in Like and Dislike conditions)
#data_fix = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/fixations_collapsed_FFA_Dislike_v1.csv') 
#data_expData = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/expdata_bin_FFA_Dislike_v1.csv') 

#for binned data
# we use v1 that is the version that doesn't have the numeration corrected
#data_fix = pd.read_csv('/Users/pradyumna/Documents/GiTs/FF2018/InputData/fixations_collapsed_FFA_Like_31.csv') 
#data_expData = pd.read_csv('/Users/pradyumna/Documents/GiTs/FF2018/output/expdata_bin_FFA_Like_31.csv') 

#for not binned data
data_fix = pd.read_csv('/Users/pradyumna/Documents/GiTs/PF2019/Output/fixations_collapsed_PF2019_More_33.csv') 
data_expData = pd.read_csv('/Users/pradyumna/Documents/GiTs/PF2019/Output/expdata_PF2019_More_33.csv') 

data_expData = data_expData.apply(pd.to_numeric, errors='ignore')
data_fix = data_fix.apply(pd.to_numeric, errors='ignore')

# Remove participants 31 , since it's excluded due to missing trial
data_expData = data_expData[data_expData.parcode != 31]
data_fix = data_fix[data_fix.parcode != 31]

In [3]:
print(data_fix.parcode.unique())
print(data_expData.parcode.unique())

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 32 33]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 32 33]


## Replace trial numerations

In [4]:
partNums = pd.unique(data_fix["parcode"])

new_fix_dataframe = data_fix.iloc[0:0]
new_expData_dataframe = data_expData.iloc[0:0]

for i in partNums:
    old_index_trial = []
    new_index_trial = []
    participant_trials = []
    
    participant_trials = data_expData.loc[data_expData['parcode'] == i]
    part_fix_trials = data_fix.loc[data_fix['parcode'] == i]
    
    old_index_trial = participant_trials['trial'].values
    new_index_trial = range(len(old_index_trial))
    
    old_index_trial_fix = part_fix_trials['trial'].unique()
    new_index_trial_fix = range(len(old_index_trial_fix))
        
    for j in new_index_trial:
            #insert new column with new trial numeration : case trialData
            trialCol2Replace = participant_trials['trial'].replace(old_index_trial[j], new_index_trial[j])
            participant_trials = participant_trials.drop(['trial'], axis=1)
            participant_trials.insert(1,'trial',trialCol2Replace)
            
            #insert new column with new trial numeration : case Fixations
            fixCol2Replace = part_fix_trials['trial'].replace(old_index_trial_fix[j], new_index_trial_fix[j]).values
            part_fix_trials = part_fix_trials.drop(['trial'], axis=1)
            part_fix_trials.insert(1,'trial',fixCol2Replace)
            
            time_trials = part_fix_trials['fix_time'].values
            
            ix = np.in1d(time_trials, 0)
            delete_fix_idx = np.where(ix)
            part_fix_trials = part_fix_trials.drop(part_fix_trials.index[delete_fix_idx])
            
    new_fix_dataframe = pd.concat([new_fix_dataframe, part_fix_trials],axis = 0)
    new_expData_dataframe = pd.concat([new_expData_dataframe, participant_trials], axis = 0)

In [5]:
new_fix_dataframe

,parcode,trial,fix_item,fix_time
0,1,0,0,171.000
1,1,0,1,868.000
2,1,0,0,266.000
3,1,0,2,429.284
4,1,1,0,235.000
5,1,1,1,314.000
6,1,1,2,680.000
7,1,1,1,696.000
8,1,1,2,770.000
9,1,1,1,1490.000


In [6]:
new_expData_dataframe

,parcode,trial,rt,choice,item_left,item_right
0,1,0,1734.284,1,110,131
1,1,1,6555.370,-1,47,50
2,1,2,3174.566,-1,50,44
3,1,3,2877.579,1,57,50
4,1,4,1806.310,1,42,50
5,1,5,3636.816,-1,108,110
6,1,6,3650.266,1,78,80
7,1,7,3078.223,-1,110,92
8,1,8,1259.268,1,50,48
9,1,9,3440.738,-1,110,101


In [7]:
new_expData_dataframe.to_csv("Output/aDDM_Data_PF2019_More_NoBin_33.csv",index = True)
new_fix_dataframe.to_csv("Output/aDDM_Fix_PF2019_More_NoBin_33.csv",index = True)

## Binning for value

In [8]:
new_expData_dataframe["item_value_0_bin"] = pd.to_numeric(pd.qcut(new_expData_dataframe["item_left"].values, 8, labels=[0, 1, 2,3,4,5,6,7]))
new_expData_dataframe["item_value_1_bin"] = pd.to_numeric(pd.qcut(new_expData_dataframe["item_right"].values, 8, labels=[0, 1, 2,3,4,5,6,7]))

In [9]:
new_expData_dataframe = new_expData_dataframe.drop(columns= ['item_left','item_right'])
new_expData_dataframe = new_expData_dataframe.rename(index=str, columns={"item_value_0_bin": "item_left","item_value_1_bin": "item_right"})

In [10]:
new_expData_dataframe.to_csv("Output/aDDM_Data_PF2019_More_YesBin_33.csv",index = True)

# Less Condition Data

In [11]:
#Input has the fixation and expdat with breaks in trial numerations (due to separation in Like and Dislike conditions)
#data_fix = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/fixations_collapsed_FFA_Dislike_v1.csv') 
#data_expData = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/expdata_bin_FFA_Dislike_v1.csv') 

#for binned data
# we use v1 that is the version that doesn't have the numeration corrected
#data_fix = pd.read_csv('/Users/pradyumna/Documents/GiTs/FF2018/InputData/fixations_collapsed_FFA_Like_31.csv') 
#data_expData = pd.read_csv('/Users/pradyumna/Documents/GiTs/FF2018/output/expdata_bin_FFA_Like_31.csv') 

#for not binned data
data_fix = pd.read_csv('/Users/pradyumna/Documents/GiTs/PF2019/Output/fixations_collapsed_PF2019_Less_33.csv') 
data_expData = pd.read_csv('/Users/pradyumna/Documents/GiTs/PF2019/Output/expdata_PF2019_Less_33.csv') 

data_expData = data_expData.apply(pd.to_numeric, errors='ignore')
data_fix = data_fix.apply(pd.to_numeric, errors='ignore')


# Remove participants 31 , since it's excluded due to missing trial
data_expData = data_expData[data_expData.parcode != 31]
data_fix = data_fix[data_fix.parcode != 31]


## Replace numerations

In [12]:
partNums = pd.unique(data_fix["parcode"])

new_fix_dataframe = data_fix.iloc[0:0]
new_expData_dataframe = data_expData.iloc[0:0]

for i in partNums:
    old_index_trial = []
    new_index_trial = []
    participant_trials = []
    
    participant_trials = data_expData.loc[data_expData['parcode'] == i]
    part_fix_trials = data_fix.loc[data_fix['parcode'] == i]
    
    old_index_trial = participant_trials['trial'].values
    new_index_trial = range(len(old_index_trial))
    
    old_index_trial_fix = part_fix_trials['trial'].unique()
    new_index_trial_fix = range(len(old_index_trial_fix))
        
    for j in new_index_trial:
            #insert new column with new trial numeration : case trialData
            trialCol2Replace = participant_trials['trial'].replace(old_index_trial[j], new_index_trial[j])
            participant_trials = participant_trials.drop(['trial'], axis=1)
            participant_trials.insert(1,'trial',trialCol2Replace)
            
            #insert new column with new trial numeration : case Fixations
            fixCol2Replace = part_fix_trials['trial'].replace(old_index_trial_fix[j], new_index_trial_fix[j]).values
            part_fix_trials = part_fix_trials.drop(['trial'], axis=1)
            part_fix_trials.insert(1,'trial',fixCol2Replace)
            
            time_trials = part_fix_trials['fix_time'].values
            
            ix = np.in1d(time_trials, 0)
            delete_fix_idx = np.where(ix)
            part_fix_trials = part_fix_trials.drop(part_fix_trials.index[delete_fix_idx])
            
    new_fix_dataframe = pd.concat([new_fix_dataframe, part_fix_trials],axis = 0)
    new_expData_dataframe = pd.concat([new_expData_dataframe, participant_trials], axis = 0)

In [13]:
new_expData_dataframe.to_csv("Output/aDDM_Data_PF2019_Less_NoBin_33.csv",index = True)
new_fix_dataframe.to_csv("Output/aDDM_Fix_PF2019_Less_NoBin_33.csv",index = True)

## Flip values (No Bin)

In [14]:
item_left_inv = 133 - new_expData_dataframe["item_left"] +40 
item_right_inv = 133 - new_expData_dataframe["item_right"]+ 40 

In [15]:
new_expData_dataframe1 = new_expData_dataframe
new_expData_dataframe1.drop(columns=['item_left', 'item_right'])
new_expData_dataframe1['item_left'] = item_left_inv
new_expData_dataframe1['item_right'] = item_right_inv

In [16]:
new_expData_dataframe1.to_csv("Output/aDDM_Data_PF2019_Less_NoBin_33_Inv.csv",index = True)

## Binning for value

In [17]:
new_expData_dataframe["item_value_0_bin"] = pd.to_numeric(pd.qcut(new_expData_dataframe["item_left"].values, 8, labels=[0, 1, 2,3,4,5,6,7]))
new_expData_dataframe["item_value_1_bin"] = pd.to_numeric(pd.qcut(new_expData_dataframe["item_right"].values, 8, labels=[0, 1, 2,3,4,5,6,7]))

In [18]:
new_expData_dataframe = new_expData_dataframe.drop(columns= ['item_left','item_right'])
new_expData_dataframe = new_expData_dataframe.rename(index=str, columns={"item_value_0_bin": "item_left","item_value_1_bin": "item_right"})

In [19]:
new_expData_dataframe.to_csv("Output/aDDM_Data_PF2019_Less_YesBin_33.csv",index = True)

## Flip values (Bin)

In [20]:
item_left_inv = 7 - new_expData_dataframe["item_left"]  
item_right_inv = 7 - new_expData_dataframe["item_right"] 

In [21]:
new_expData_dataframe.drop(columns=['item_left', 'item_right'])
new_expData_dataframe['item_left'] = item_left_inv
new_expData_dataframe['item_right'] = item_right_inv

In [22]:
new_expData_dataframe.to_csv("Output/aDDM_Data_PF2019_Less_YesBin_33_Inv.csv",index = True)